In [1]:
#Exercise 1: Importing data
import pandas as pd
Duncan_Data = pd.read_csv('Duncan_Data.csv')
Duncan_Data.head()

,profession,type,income,education,prestige
0,accountant,prof,62,86,82
1,pilot,prof,72,76,83
2,architect,prof,75,92,90
3,author,prof,55,90,76
4,chemist,prof,64,86,90


In [10]:
#Exercise 1a
import statsmodels.formula.api as sm
model1 = sm.ols('prestige~income',data=Duncan_Data).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:               prestige   R-squared:                       0.702
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     101.3
Date:                Fri, 16 Dec 2022   Prob (F-statistic):           7.14e-13
Time:                        15:16:15   Log-Likelihood:                -191.38
No. Observations:                  45   AIC:                             386.8
Df Residuals:                      43   BIC:                             390.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.4566      5.190      0.473      0.6

In [7]:
#Exercise 1b
#calculate the cook's D value
CooksD = model1.get_influence().cooks_distance
#calculate the sample size
n = len(Duncan_Data)
#add a variable to the data containing the Cooks D
Duncan_Data['Outlier'] = CooksD[0] > 4/n #note we use CooksD[0] because we only want the first array, the second array contains p-values but we're not using those
#Inspect the outliers
print(Duncan_Data[Duncan_Data.Outlier==True])


   profession  type  income  education  prestige  Outlier
5    minister  prof      21         84        87     True
15  conductor    wc      76         34        38     True


In [9]:
#Exercise 1c
#Remove the outliers
Duncan_Data2 = Duncan_Data[Duncan_Data.Outlier==False]

#un the regression again
model2 = sm.ols('prestige~income',data=Duncan_Data2).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:               prestige   R-squared:                       0.841
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     216.7
Date:                Fri, 16 Dec 2022   Prob (F-statistic):           5.81e-18
Time:                        15:16:05   Log-Likelihood:                -169.52
No. Observations:                  43   AIC:                             343.0
Df Residuals:                      41   BIC:                             346.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.7435      3.901     -0.703      0.4

In [11]:
#Exercise 2a
houses = pd.read_csv('housing_price_ge19.csv')
houses.head()



,Number,Price,Lot_Size,Waterfront,Age,Land_Value,New_Construct,Central_Air,Fuel_Type,Heat_Type,Sewer_Type,Living_Area,Pct_College,Bedrooms,Fireplaces,Bathrooms,Rooms
0,1,132500,0.09,0,42.0,50000.0,0,0,Electric,Electric,Private,906,35,2,1.0,1.0,5
1,2,181115,0.92,0,0.0,22300.0,0,0,Gas,Hot_Water,Private,1953,51,3,0.0,2.5,6
2,3,109000,0.19,0,133.0,7300.0,0,0,Gas,NaN,Public,1944,51,4,1.0,1.0,8
3,4,155000,0.41,0,13.0,18700.0,0,0,Gas,Hot_Air,Private,1944,51,3,1.0,1.5,5
4,5,86060,0.11,0,0.0,15000.0,1,1,Gas,Hot_Air,Public,840,51,2,0.0,1.0,3


In [12]:
#check which variables have missing data
houses.isnull().sum()

Number             0
Price              0
Lot_Size           0
Waterfront         0
Age               80
Land_Value        81
New_Construct      0
Central_Air        0
Fuel_Type          0
Heat_Type         64
Sewer_Type        12
Living_Area        0
Pct_College        0
Bedrooms           0
Fireplaces       398
Bathrooms          0
Rooms              0
dtype: int64

In [15]:
#remove the missing values and create dummies
houses_m = houses.dropna()
houses_m = pd.get_dummies(houses_m,
                          drop_first=True) #this will automatically drop the first category

,Number,Price,Lot_Size,Waterfront,Age,Land_Value,New_Construct,Central_Air,Living_Area,Pct_College,...,Bathrooms,Rooms,Fuel_Type_Electric,Fuel_Type_Gas,Fuel_Type_Oil,Heat_Type_Electric,Heat_Type_Hot_Air,Heat_Type_Hot_Water,Sewer_Type_Private,Sewer_Type_Public
0,1,132500,0.09,0,42.0,50000.0,0,0,906,35,...,1.0,5,1,0,0,1,0,0,1,0
1,2,181115,0.92,0,0.0,22300.0,0,0,1953,51,...,2.5,6,0,1,0,0,0,1,1,0
3,4,155000,0.41,0,13.0,18700.0,0,0,1944,51,...,1.5,5,0,1,0,0,1,0,1,0
4,5,86060,0.11,0,0.0,15000.0,1,1,840,51,...,1.0,3,0,1,0,0,1,0,0,1
7,8,170000,1.21,0,23.0,14600.0,0,0,1662,35,...,1.5,9,0,0,1,0,1,0,1,0


In [ ]:
#Exercise 2b
#Run the regression with missing values removed

#because there are many variables, I'm gonna shorten the code (if you want, you can also shorten further using a different method found on the statsmodel website)
houses_m = houses_m.drop(columns='Number') #drop the first variable
column_names = list(houses_m.columns) #store the column names in a seperate index
new_columns = [chr(ord('a') + i) for i in range(len(houses_m.columns))] #this creates a loop giving a, b, c, d etc
houses_m.columns=new_columns #with replace the columns with letters

#run the model
model = sm.ols('a~b+c+d+e+f+g+h+i+j+k+l+m+n+o+p+q+r',data=houses_m).fit()

#print results
column_names[0]="intercept" #replace the first word with the word 'intercept'
print(model.summary(xname=column_names)) #print the results, but replace the variable names with the original column names


In [ ]:
#Exercise 2c

#replace the missing variables
houses = pd.read_csv('housing_price_ge19.csv')
#Create dummy variables, including for the missing variables
houses = pd.get_dummies(houses,drop_first=True, dummy_na = True, )
#impute the remaining data
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
houses_imp = IterativeImputer().fit_transform(houses)
houses_imp = pd.DataFrame(houses_imp, columns=houses.columns)

#run the model
#shorten the code again
houses_imp = houses_imp.drop(columns='Number') #drop the first variable
column_names = list(houses_imp.columns) #store the column names in a seperate index
new_columns = [chr(ord('a') + i) for i in range(len(houses_imp.columns))] #this creates a loop giving a, b, c, d etc
houses_imp.columns=new_columns #with replace the columns with letters

model = sm.ols('a~b+c+d+e+f+g+h+i+j+k+l+m+n+o+p+q+r+s+t+u',data=houses_imp).fit()

#print results
column_names[0]="intercept" #replace the first word with the word 'intercept'
print(model.summary(xname=column_names)) #print the results, but replace the variable names with the original column names


